# Demo Notebook

A notebook describing the basics for connecting via python to oracle. 

**First, ensure that you installed cx_Oracle with:**

`python -m pip install cx_Oracle --upgrade`

## Get the libraries

In [5]:
import cx_Oracle

## Get Basic Client

see here: https://www.oracle.com/database/technologies/instant-client/winx64-64-downloads.html

In [6]:
cx_Oracle.init_oracle_client('/opt/oracle/instantclient_23_3') #add here the path to the instantclient

## Set the credentials

In [7]:
username="a11907564"  #db user name
password="dbs24" #db password 
con_string="oracle19.cs.univie.ac.at:1521/orclcdb"

## Connect

In [8]:
#consider to use with in order to avoid open connections
connection = cx_Oracle.connect(user=username, password=password,
                               dsn=con_string,
                               encoding="UTF-8")

Check if connection is now up!

In [57]:
#create a table
curs=connection.cursor()
create_stat="create table myTestTable(myId INTEGER)" #just an example: always use a PK
curs.execute(create_stat)
connection.commit()

DatabaseError: ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/

In [58]:
#insert into
myIdValue=4
create_stat="insert into myTestTable (myId) VALUES (:myIdValue)"
curs.execute(create_stat,myIdValue=myIdValue)
connection.commit()

In [59]:
#select
curs.execute("select * from myTestTable")
res = curs.fetchall()

In [60]:
str(res)  #result

'[(4,), (4,), (4,)]'

In [61]:
[x[0] for x in res]

[4, 4, 4]

# Read from CSV

In [62]:
import pandas as pd

In [63]:
example_data=pd.read_csv("ExampleData.csv",sep=";")

In [64]:
example_data.head()

,Id,Name
0,9,Ann
1,10,Bob
2,11,Carol


In [65]:
#create a table
create_stat="create table myTestTable2(myId INTEGER,myName VARCHAR(30))" #just an example: always use a PK
curs.execute(create_stat)
connection.commit()

DatabaseError: ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/

In [66]:
def add_tuple(m_tuple):
    create_stat="insert into myTestTable2 (myId,myName) VALUES (:myId,:myName)"
    curs.execute(create_stat,myId=m_tuple.Id,myName=m_tuple.Name)
    connection.commit()

In [67]:
example_data.apply(lambda x: add_tuple(x),axis=1)  #use for instead of apply if that is easier for your!
print("done with inserting....")

done with inserting....


In [68]:
curs.execute("select * from myTestTable2")
res = curs.fetchall()
res

[(9, 'Ann'),
 (10, 'Bob'),
 (11, 'Carol'),
 (9, 'Ann'),
 (10, 'Bob'),
 (11, 'Carol'),
 (9, 'Ann'),
 (10, 'Bob'),
 (11, 'Carol')]

# DB Triggers

execute the Triggers from Aufgabe 3-2.

Can be found in db_triggers.sql

In [82]:
import os


# Path to the SQL file
sql_file_path = os.path.join(os.path.dirname('db_triggers.sql'), 'db_triggers.sql')

try:
    # Read the SQL file
    with open(sql_file_path, 'r') as file:
        sql_commands = file.read()

    # Execute the SQL commands
    for command in sql_commands.split(';'):  # Split commands if needed
        command = command.strip()
        if command:  # Ignore empty commands
            curs.execute(command)
            print(f"Executed: {command}")

    # Commit changes
    connection.commit()
    print("All commands executed successfully.")

except Exception as e:
    print(f"Error: {e}")

All commands executed successfully.


# Aufgabe 3-3

1. Verbindung zur Datenbank

In [ ]:
import cx_Oracle

# first wie want to establish a connection to the database (just in case this hasn't allready been done)

connection = cx_Oracle.connect(user=username, password=password,
                               dsn=con_string,
                               encoding="UTF-8")
curs = connection.cursor()

print("Verbindung erfolgreich!")



#then we want to make sure that the physical layout of the DB is established. The according Requests can be found in the File db_triggers.sql

# Path to the SQL file
sql_file_path = 'db_trigger.sql' 

try:
    # Read the SQL file
    with open(sql_file_path, 'r') as file:
        sql_commands = file.read()
        print(file)

    # Execute the SQL commands
    for command in sql_commands.split(';'):  # Split commands if needed
        command = command.strip()
        if command:  # Ignore empty commands
            try:
                curs.execute(command)
                print(f"Executed: {command}")
            except Exception as e:
                print(f"Error : {e}")

    # Commit changes
    connection.commit()
    print("All commands executed successfully.")

except Exception as e:
    print(f"Error: {e}")



Verbindung erfolgreich!
<_io.TextIOWrapper name='db_triggers.sql' mode='r' encoding='utf-8'>
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.

2. Generierung von Testdaten für die Kunden Tabelle


In [20]:
import random

# Kunden-Tabelle mit 1000 Einträgen befüllen

for i in range(1, 1001):
    kunden_id = i
    name = f'Kunde_{i}'
    adresse = f'Straße {random.randint(1, 100)}, Stadt {random.randint(1, 10)}'
    telefonnummer = f'01234567{random.randint(100, 999)}'
    email = f'kunde_{i}@example.com'
    try:
        curs.execute("""
            INSERT INTO Kunden (KundenID, Name, Adresse, Telefonnummer, Email)
            VALUES (:1, :2, :3, :4, :5)
        """, (kunden_id, name, adresse, telefonnummer, email))
    except Exception as e:
        print(f"Error: {e}")

connection.commit()
print("Kunden-Daten erfolgreich eingefügt!")


Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-hel

3. Generierung großer Datensätze für andere Tabellen


In [17]:
# Fahrzeuge-Tabelle mit 500 Einträgen
for i in range(1, 501):
    fahrzeug_id = i
    kennzeichen = f'ABC-{i:04d}'
    modell = random.choice(['SUV', 'Kleinwagen', 'Limousine'])
    hersteller = random.choice(['Hersteller A', 'Hersteller B'])
    baujahr = random.randint(2000, 2023)
    status = random.choice(['verfügbar', 'vermietet', 'in Wartung'])
    try:
        curs.execute("""
            INSERT INTO Fahrzeuge (FahrzeugID, Kennzeichen, Modell, Hersteller, Baujahr, Status)
            VALUES (:1, :2, :3, :4, :5, :6)
        """, (fahrzeug_id, kennzeichen, modell, hersteller, baujahr, status))
    except Exception as e:
        print(f"Error: {e}")

connection.commit()
print("Fahrzeug-Daten erfolgreich eingefügt!")


Fahrzeug-Daten erfolgreich eingefügt!


In [28]:
# Reservierungen-Tabelle mit 10.000 Einträgen
for i in range(1, 10):
    reservierungs_id = i
    kunden_id = random.randint(1, 1000)
    fahrzeug_id = random.randint(1, 500)
    startdatum = f'2025-01-{random.randint(1, 28)}'
    enddatum = f'2025-01-{random.randint(29, 31)}'
    kosten = round(random.uniform(50, 500), 2)
    try:
        curs.execute("""
            INSERT INTO Reservierungen (ReservierungsID, KundenID, FahrzeugID, Startdatum, Enddatum, Kosten)
            VALUES (:1, :2, :3, TO_DATE(:4, 'YYYY-MM-DD'), TO_DATE(:5, 'YYYY-MM-DD'), :6)
        """, (reservierungs_id, kunden_id, fahrzeug_id, startdatum, enddatum, kosten))
    except Exception as e:
        print(f"Error: {e}")
    
connection.commit()
print("Reservierungs-Daten erfolgreich eingefügt!")


Error: ORA-04098: trigger 'A11907564.UPDATE_VEHICLE_STATUS' is invalid and failed re-validation
Help: https://docs.oracle.com/error-help/db/ora-04098/
Error: ORA-04098: trigger 'A11907564.UPDATE_VEHICLE_STATUS' is invalid and failed re-validation
Help: https://docs.oracle.com/error-help/db/ora-04098/
Error: ORA-04098: trigger 'A11907564.UPDATE_VEHICLE_STATUS' is invalid and failed re-validation
Help: https://docs.oracle.com/error-help/db/ora-04098/
Error: ORA-04098: trigger 'A11907564.UPDATE_VEHICLE_STATUS' is invalid and failed re-validation
Help: https://docs.oracle.com/error-help/db/ora-04098/
Error: ORA-04098: trigger 'A11907564.UPDATE_VEHICLE_STATUS' is invalid and failed re-validation
Help: https://docs.oracle.com/error-help/db/ora-04098/
Error: ORA-04098: trigger 'A11907564.UPDATE_VEHICLE_STATUS' is invalid and failed re-validation
Help: https://docs.oracle.com/error-help/db/ora-04098/
Error: ORA-04098: trigger 'A11907564.UPDATE_VEHICLE_STATUS' is invalid and failed re-validatio

4. Daten überprüfen

In [22]:
# Anzahl der Tupel in jeder Tabelle prüfen
for table in ['Kunden', 'Fahrzeuge', 'Reservierungen']:
    curs.execute(f"SELECT COUNT(*) FROM {table}")
    count = curs.fetchone()[0]
    print(f"Tabelle {table}: {count} Tupel")


Tabelle Kunden: 1000 Tupel
Tabelle Fahrzeuge: 500 Tupel
Tabelle Reservierungen: 0 Tupel


5. Sinnvolle Querries

Query 1: Durchschnittliche Kosten pro Fahrzeugmodell


In [24]:
query1 = """
SELECT Modell, AVG(Kosten) AS Durchschnittskosten
FROM Fahrzeuge
JOIN Reservierungen ON Fahrzeuge.FahrzeugID = Reservierungen.FahrzeugID
GROUP BY Modell
HAVING AVG(Kosten) > 200
"""
curs.execute(query1)
for row in curs.fetchall():
    print(f"Output: {row}")


Query 2: Top-Kunden mit den meisten Reservierungen

In [25]:
query2 = """
SELECT Kunden.Name, COUNT(Reservierungen.ReservierungsID) AS AnzahlReservierungen
FROM Kunden
JOIN Reservierungen ON Kunden.KundenID = Reservierungen.KundenID
GROUP BY Kunden.Name
ORDER BY AnzahlReservierungen DESC
FETCH FIRST 10 ROWS ONLY
"""
curs.execute(query2)
for row in curs.fetchall():
    print(row)


Query 3: Fahrzeuge mit aktuellen Reservierungen

In [26]:
query3 = """
SELECT Fahrzeuge.FahrzeugID, Fahrzeuge.Kennzeichen, Reservierungen.Startdatum, Reservierungen.Enddatum
FROM Fahrzeuge
JOIN Reservierungen ON Fahrzeuge.FahrzeugID = Reservierungen.FahrzeugID
WHERE Reservierungen.Startdatum <= SYSDATE AND Reservierungen.Enddatum >= SYSDATE
"""
curs.execute(query3)
for row in curs.fetchall():
    print(row)


6. Optimierung durch Indexing

In [27]:
# Index hinzufügen
curs.execute("CREATE INDEX idx_fahrzeugid ON Reservierungen(FahrzeugID)")

# Performance vergleichen
curs.execute("EXPLAIN PLAN FOR SELECT * FROM Reservierungen WHERE FahrzeugID = 101")
for row in curs.fetchall():
    print(row)


InterfaceError: not a query

# Close Connection

In [89]:
curs.close()
connection.close()